## Problem 2: Visualize an interactive map (10 points)

Create a nice interactive map using the skills you leared in lesson 5: interactive maps using Folium (or some other suitable package - feel free to experiment!). Write your code into a notebook file (`.ipynb`) or a python script file (.py) and store the output map(s) in `.html` format under the `docs` folder.

**Topic of the map:**
- You can select the topic of the map freely.
- This map should not be only an interactive version of your submission for problem 1 ;). Create something new!
- Feel free to adapt examples provided in this course!

**Criteria:**
- The map should have multiple layers on it and/or present an output of (simple) data analysis (something beyond plotting raw data on a map).
- Consider [good cartographic practices](https://www.gislounge.com/ten-things-to-consider-when-making-a-map/)
- The map should demonstrate skills learned during lesson 5: interactive maps and throughout this course. You can also take advantage of the [Folium example gallery](https://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/) for further ideas.
- Think about including additional interactive elements, such as popup text in the map.

**Output:**
- Remember to commit the code and input data (or at least a link to input data)
- Save your map(s) in .html format in the **`docs`** folder

The addresses for the metro stations were derived from [the city of Helsinki's website](https://www.hel.fi/hkl/fi/metrolla/metroasemat/).
The data for the 15 minutes walking time distance buffers around the metro stations were derived from [Helsinki Region Infoshare](https://hri.fi/data/en_GB/dataset/kavely-ja-pyorailyaikavyohykkeet-paakaupunkiseudun-asemille). And the data containing the HSL public transport lines were derived from the [HSL public trandsport dataset](https://hri.fi/data/en_GB/dataset/hsl-n-linjat) and the metro line was selected.

In [1]:
# Import modules
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point, MultiPoint, LineString
from pyproj import CRS
import folium
import requests
import geojson
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt

# Import the metro shapefile
metro = gpd.read_file("data/Metro.shp")

In [2]:
# Load population grid data
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format).
# Available feature types in this particular data source: http://geo.stat.fi/geoserver/vaestoruutu/wfs?service=wfs&version=2.0.0&request=describeFeatureType
params = dict(service='WFS',
              version='2.0.0',
              request='GetFeature',
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018',
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson
data = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

# Check the data
data.head()

,geometry,index,asukkaita,asvaljyys,ika0_9,ika10_19,ika20_29,ika30_39,ika40_49,ika50_59,ika60_69,ika70_79,ika_yli80
0,MULTIPOLYGON Z (((25476499.999 6674248.999 0.0...,3342,108,45,11,23,6,7,26,17,8,6,4
1,MULTIPOLYGON Z (((25476749.997 6674498.998 0.0...,3503,273,35,35,24,52,62,40,26,25,9,0
2,MULTIPOLYGON Z (((25476999.994 6675749.004 0.0...,3660,239,34,46,24,24,45,33,30,25,10,2
3,MULTIPOLYGON Z (((25476999.994 6675499.004 0.0...,3661,202,30,52,37,13,36,43,11,4,3,3
4,MULTIPOLYGON Z (((25476999.994 6675249.005 0.0...,3662,261,30,64,32,36,64,34,20,6,3,2


In [3]:
# Assign crs and reproject to WGS 84 (EPSG: 4326)
data.crs = CRS.from_epsg(3879)
data = data.to_crs(epsg=4326)
print(data.crs)

epsg:4326


In [4]:
# Rename the population column
data = data.rename(columns={'asukkaita':'pop18'})

# Create a geoid column
data['geoid'] = data.index.astype(str)

# Drop unnecessary columns
data = data[['geoid', 'pop18', 'geometry']]

# Check prepared dataframe
data.head()

,geoid,pop18,geometry
0,0,108,"MULTIPOLYGON Z (((24.57654 60.18042 0.00000, 2..."
1,1,273,"MULTIPOLYGON Z (((24.58102 60.18267 0.00000, 2..."
2,2,239,"MULTIPOLYGON Z (((24.58538 60.19391 0.00000, 2..."
3,3,202,"MULTIPOLYGON Z (((24.58541 60.19166 0.00000, 2..."
4,4,261,"MULTIPOLYGON Z (((24.58544 60.18942 0.00000, 2..."


In [5]:
# Read textfile containing the addresses of the Helsinki metro stations
metrostations = pd.read_csv("data/metro_stations.txt", sep=';', header=0)

# Check dataframe head
metrostations.head()

,station,addr
0,Matinkylä,"Suomenlahdentie 1, 02230 Espoo"
1,Niittykumpu,"Niittykatu 2, 02200 Espoo"
2,Urheilupuisto,"Jousenpuistonkatu 2, 02200 Espoo"
3,Tapiola,"Merituulentie, 02100 Espoo"
4,Aalto-yliopisto,"Otaniementie 12, 02150 Espoo"


In [6]:
# Read shapefile containing 15 mins walking time buffers around the metro stations
walkdist15 = gpd.read_file("data/15min_Walk_metro.shp")

# Reproject layer
walkdist15 = walkdist15.to_crs(epsg=4326)

# Check the dataframe head
walkdist15.head()

,Asema,Arvo,Unit,Area_VM,geometry
0,Kalasatama,15.0,min_Walk,2.060425e+06,"MULTIPOLYGON Z (((24.99273 60.18500 0.00000, 2..."
1,Vuosaari,15.0,min_Walk,3.170834e+06,"POLYGON Z ((25.15374 60.20090 0.00000, 25.1530..."
2,Myllypuro,15.0,min_Walk,3.244018e+06,"POLYGON Z ((25.08060 60.23486 0.00000, 25.0803..."
3,Rautatientori,15.0,min_Walk,4.018450e+06,"MULTIPOLYGON Z (((24.93662 60.18118 0.00000, 2..."
4,Helsinki,15.0,min_Walk,3.896943e+06,"MULTIPOLYGON Z (((24.94648 60.18036 0.00000, 2..."


In [7]:
# Spatial join between the population data and the walking distance buffers
join = gpd.sjoin(walkdist15, data, how = 'inner', op = 'contains')

# Calculate how many people live within a 15 minute walking distance from each metro station
walkdist_pop = join.groupby('Asema').sum()
walkdist_pop = walkdist_pop['pop18']

# Using a for loop to print the result
for i in range(len(walkdist_pop)):
    print(walkdist_pop[i], "people live within a 15 minute walking distance from", walkdist_pop.index[i], ".")

2251 people live within a 15 minute walking distance from Aalto-yliopisto .
26042 people live within a 15 minute walking distance from Hakaniemi .
14745 people live within a 15 minute walking distance from Helsingin yliopisto .
17063 people live within a 15 minute walking distance from Helsinki .
10074 people live within a 15 minute walking distance from Herttoniemi .
6987 people live within a 15 minute walking distance from Itäkeskus .
5771 people live within a 15 minute walking distance from Kalasatama .
26341 people live within a 15 minute walking distance from Kamppi .
1695 people live within a 15 minute walking distance from Keilaniemi .
1999 people live within a 15 minute walking distance from Koivusaari .
13663 people live within a 15 minute walking distance from Kontula .
2508 people live within a 15 minute walking distance from Kulosaari .
11661 people live within a 15 minute walking distance from Lauttasaari .
14421 people live within a 15 minute walking distance from Matinky

In [8]:
# Merge the walking distance buffers
walkdist15 = walkdist15.merge(walkdist_pop, on="Asema", how="left")

# Rename the column
walkdist15.rename(columns = {"pop18": "pop"}, inplace = True)

# Drop unnecessary columns
walkdist15 = walkdist15[['Asema', 'Area_VM', 'geometry', 'pop']]

# Check dataframe head
walkdist15.head()

,Asema,Area_VM,geometry,pop
0,Kalasatama,2.060425e+06,"MULTIPOLYGON Z (((24.99273 60.18500 0.00000, 2...",5771
1,Vuosaari,3.170834e+06,"POLYGON Z ((25.15374 60.20090 0.00000, 25.1530...",17474
2,Myllypuro,3.244018e+06,"POLYGON Z ((25.08060 60.23486 0.00000, 25.0803...",10474
3,Rautatientori,4.018450e+06,"MULTIPOLYGON Z (((24.93662 60.18118 0.00000, 2...",23468
4,Helsinki,3.896943e+06,"MULTIPOLYGON Z (((24.94648 60.18036 0.00000, 2...",17063


In [9]:
# Geocode the addresses using Nominatim
stations = geocode(metrostations['addr'], provider='nominatim', user_agent='autogis_HP', timeout=20)

In [10]:
# Add the station name to the geodataframe
stations['name'] = metrostations['station']

# Check dataframe head
stations.head()

,geometry,address,name
0,POINT (24.73820 60.15976),"Chapple, 1, Suomenlahdentie, Matinkylän keskus...",Matinkylä
1,POINT (24.75802 60.16699),"Niittykatu, Niittykumpu, Suur-Tapiola, Espoo, ...",Niittykumpu
2,POINT (24.78039 60.17436),"Sidewalk Urheilupuisto, 2, Jousenpuistonkatu, ...",Urheilupuisto
3,POINT (24.80370 60.17490),"Merituulentie, Tapiolan keskus, Tapiola, Suur-...",Tapiola
4,POINT (24.82492 60.18491),"K-Market Otaniemi, 12, Otaniementie, Otaniemi,...",Aalto-yliopisto


In [11]:
# Points and line to GeoJSON
#stations_gjson = folium.features.GeoJson(stations, name="Metro stations")
metro_gjson = folium.features.GeoJson(metro, name="Metro route")
#walkdist_gjson = folium.features.GeoJson(walkdist15, name="15 minutes walking time")

In [14]:
# Create the interactive map

# Create a Map instance
m = folium.Map(location=[60.20, 24.95], zoom_start=11, tiles="cartodbpositron", control_scale=True)

# Choropleth for the population data
folium.Choropleth(
    geo_data=data,
    data=data,
    name='Population',
    columns=['geoid', 'pop18'],
    key_on='feature.id',
    fill_color="YlOrRd",
    line_weight=0
    ).add_to(m)

# Adding the features
# Metro station
folium.features.GeoJson(stations,
                        name='Metro stations',
                        style_function=lambda x: {'color':'white','fillColor':'red','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['name'],
                                                                aliases = ['Station'],
                                                                labels=True,
                                                                sticky=False)
                       ).add_to(m)


# Population living within 15 minutes walking distance to the metro station
folium.features.GeoJson(walkdist15,
                        name='Area of 15 mins walking time',
                        style_function=lambda x: {'color':'green','fillColor':'black','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=(['Asema', 'pop']),
                                                                aliases = (['Station', 'Population']),
                                                                labels=True,
                                                                sticky=False)
                       ).add_to(m)

# Adding the different layers of the metro line, stations, and the walking distance areas
metro_gjson.add_to(m)


# Create a layer control object and add it to our map instance
folium.LayerControl().add_to(m)

# Print map
m

In [16]:
# Save map as html
outfp = "docs/Helsinki_metroline.html"
m.save(outfp)